In [8]:
from multiprocessing import Process, Manager
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
import tiktoken
import json
import numpy as np
import requests
from retry import retry
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
config = dotenv_values('.env')

In [9]:
MODEL_NAME = 'qwen-max'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [18]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}
### 问题:
{question}
### 选项:
{options}
"""

    return prompt

In [19]:
def extract(input_text):
    ans_pattern = re.compile(r"答案是：(.*)", re.S)

    # 使用 findall 方法找到所有匹配的字符串
    problems = ans_pattern.findall(input_text)
    if len(problems) == 0:
        return '提取失败'

    # 提取答案中出现的所有可能的答案字母
    possible_answers = "ABCDEFG"  # 假设可能的答案字母是 A, B, C, D
    
    answers = [match for match in problems[0] if match in possible_answers]

#     print(answers)  # 输出：['A']
#     ans_pattern = re.compile(r"答案是：(.)", re.S)
#     problems = ans_pattern.findall(input_text)
    if len(answers) == 0:
        return '提取失败'
        
    return answers[0]

In [20]:
def produce(data, MODEL_NAME, return_list, pid):
    tqdm1 = tqdm
    for task in tqdm1(data):
            problem = task['problem']
            for question in task['questions']:

                prompt = get_prompt(problem, 
                                    question['question'],
                                    question['options'],
                                    )

                response = call_qwen_api(MODEL_NAME, prompt)
                question['qwen_respond'] = response
                extract_response = extract(response)
                question[MODEL_NAME] = extract_response
                
                # try:
                #     attemp = 1
                #     while attemp <= 3:
                #         #print(f'try：{attemp}')
                #         extract_response = extract(response)
                #         question[MODEL_NAME] = extract_response
                #         if question[MODEL_NAME] == '':
                #             print(f'wtrong response：{response}')
                #             attemp += 1
                #             continue
                #         else:
                #             attemp = 4
                #     # if pid == 0:
                #     #     pprint(extract_response)
                #     # break
                # except:
                #     pass
            return_list.append(task)

In [21]:
def main(data, ofn):
    # 如果输出文件已经存在，则不进行处理
    # if os.path.exists(ofn):
    #     pass

    # 设置进程池大小
    POOL_SIZE = 5
    #data = []

    # 将数据分割成POOL_SIZE个子列表
    datas = [data[i::POOL_SIZE] for i in range(POOL_SIZE)]

    # 使用Manager创建一个共享对象
    with Manager() as manager:
        # 创建一个共享列表，用于存储所有进程的结果
        return_list = manager.list()

        # 创建进程列表
        producers = []
        for i in range(POOL_SIZE):
            # 创建每个进程，传递数据子集、模型名称、结果列表和进程编号
            p = Process(target=produce,
                        args=(datas[i],  # 数据子集
                              MODEL_NAME,  # 模型名称
                              return_list,  # 共享结果列表
                              i,  # 进程编号
                              )
                        )
            producers.append(p)

        # 启动所有进程
        for p in producers:
            p.start()

        # 等待所有进程完成
        for p in producers:
            p.join()

        # 输出结果列表的长度
        print(len(return_list))

        # 写入输出文件
        with open(ofn, 'w') as writer:
            for sample in return_list:
                # 将结果写入文件
                writer.write(json.dumps(sample, ensure_ascii=False))
                writer.write('\n')

    # 输出任务完成的消息
    print("All tasks finished!")

    # 调用评估函数
    evaluate(ofn)

In [22]:
def evaluate(ofn):
    data = []
    with open(ofn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    pse = 0
    cnt = 0
    tot = 0
    for task in data:
        for question in task['questions']:
            
            if MODEL_NAME in question:
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1

    print(cnt, tot, cnt/tot, pse)

In [5]:
def read_file(ifn):
    # 读取输入文件
    data=[]
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    # 打乱数据顺序
    #random.shuffle(data)
    return data

In [25]:
if __name__ == '__main__':

    a = extract("""根据欧几里得算法，逐步解析计算两个数6和7的最大公约数（gcd）的步骤如下：

1. 判断6和7是否相等：不相等。
2. 判断6和7大小关系，7 > 6，所以用更大的数7减去较小的数6得到结果1。
3. 现在计算6和1的最大公约数。
4. 6 > 1，根据算法用更大的数6减去较小的数1得到结果5。
5. 再计算5和1的最大公约数。
6. 5 > 1，用5减去1得到结果4。
7. 再计算4和1的最大公约数。
8. 4 > 1，用4减去1得到结果3。
9. 再计算3和1的最大公约数。
10. 3 > 1，用3减去1得到结果2。
11. 再计算2和1的最大公约数。
12. 2 > 1，用2减去1得到结果1。
13. 最后计算1和1的最大公约数，两数相等，gcd即为这两个数，也就是1。

因此，6和7的最大公约数是1。

答案是：应该是阿飞.""")
    # data = read_file('data/round1_train_data.jsonl')
    # main(data[:2],'result/qwen.jsonl')

    vaild_file = read_file('data/valid_data_500.jsonl')
    main(vaild_file, 'result/qwen_vaild.jsonl')

  1%|          | 1/100 [01:56<3:12:26, 116.63s/it]

Request id: 11dc4bce-41fb-95d5-8373-64ea6ea579ae, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


  4%|▍         | 4/100 [03:39<1:11:16, 44.55s/it] 

Request id: ed6c79a1-510e-9984-8441-aa71855504be, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


  6%|▌         | 6/100 [04:35<1:01:32, 39.28s/it]

...

Request id: 8dc9e050-6294-9fe5-842e-593c4a28b6d3, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: db44897a-a163-97d4-b4fe-81f1864c3bd1, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


 92%|█████████▏| 92/100 [1:12:54<04:07, 30.94s/it]

Request id: f2b81cd8-d6d5-9fce-8f82-ab3cd96c4d1f, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


100%|██████████| 100/100 [1:39:56<00:00, 59.97s/it]


407
All tasks finished!
1697 2247 0.7552291944815309 0


In [1]:
def validate_qwen_answers(file):
    validated_problem_sets = []
    
    for data in file:
        if 'promble' in data:
            data['problem'] = data.pop('promble')
        problem = data['problem']
        questions = data['questions']
        _id = data['id']
        
        validated_questions = []
        for question in questions:
            answer = question['answer']
            # 检查答案是否为大写字母
            if len(answer) == 1 and answer.isupper() and question[MODEL_NAME] == question['answer']:
                validated_questions.append(question)
            else:
                print(f"Invalid answer '{answer}' for question: {question['question']}")
        
        # 创建一个新的问题集字典，只包含有效的问题
        validated_problem_set = {
            'problem': problem,
            'questions': validated_questions,
            'id': _id
        }
        validated_problem_sets.append(validated_problem_set)
    
    return validated_problem_sets

In [2]:
def validate_answers(file):
    validated_problem_sets = []
    
    for data in file:
        if 'promble' in data:
            data['problem'] = data.pop('promble')
        problem = data['problem']
        questions = data['questions']
        _id = data['id']
        
        validated_questions = []
        for question in questions:
            answer = question['answer']
            # 检查答案是否为大写字母
            if len(answer) == 1 and answer.isupper():
                validated_questions.append(question)
            else:
                print(f"Invalid answer '{answer}' for question: {question['question']}")
        
        # 创建一个新的问题集字典，只包含有效的问题
        validated_problem_set = {
            'problem': problem,
            'questions': validated_questions,
            'id': _id
        }
        validated_problem_sets.append(validated_problem_set)
    
    return validated_problem_sets

In [3]:
import json
def write_jsonl(results, filename):
    with open(f'data/{filename}.jsonl', 'w',encoding='utf-8') as outfile:
        for entry in results:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

In [ ]:
file = read_file('result/qwen_vaild.jsonl')
vaild_file = validate_qwen_answers(file)
write_jsonl(vaild_file,"qwen_valid_data_500")

In [28]:
def g8_answers(file):
    validated_problem_sets = []
    for data in file:

        problem = data['problem']
        questions = data['questions']
        _id = data['id']
        validated_questions = []
        for question in questions:
            if 'solution' in question:
                s = question['answer']
                answer = f'答案是：{s}'
                question['qwen_respond'] = question.pop('solution') + answer
            validated_questions.append(question)
        # 创建一个新的问题集字典，只包含有效的问题
        validated_problem_set = {
            'problem': problem,
            'questions': validated_questions,
            'id': _id
        }
        validated_problem_sets.append(validated_problem_set)
    return validated_problem_sets